In [17]:
include("../runsetup_temp.jl");

In [20]:
using LinearAlgebra

In [18]:
# Training data
start = 2018, 1, 2
dayslookback = 7
ndays = 365-dayslookback-1
predicttype = :temp
offset = 1

X, y = build_multi_data(mdata, predicttype, start, dayslookback, 
    ndays; offset=offset);

(train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, X, y);

In [4]:
# training
@time begin
grid = IAI.GridSearch(
    IAI.OptimalTreeRegressor(
        random_seed=123,
        max_depth = 5,
        minbucket = 10,
        show_progress=false
    ),
    show_progress=false  
)
IAI.fit!(grid, train_X, train_y)
end

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAILicensing C:\Users\iai\builds\InterpretableAI\SysImgBuilder\.julia\packages\IAILicensing\x1CT6\src\precompile.jl:19
┌ Warning: Interpretable AI license expires soon: 2020-12-31. If you need to renew, please send us the following machine ID:
│ 1d23b3eee464c21ff412aa6a5b247b9f0543ded03e05d39d863fbfc9241e7a1f
└ @ IAILicensing C:\Users\iai\builds\InterpretableAI\SysImgBuilder\.julia\packages\IAILicensing\x1CT6\src\precompile.jl:29


561.722282 seconds (27.55 M allocations: 1.525 GiB, 0.19% gc time)


All Grid Results:

│ Row │ show_progress │ cp         │ train_score │ valid_score │
│     │ Bool          │ Float64    │ Float64     │ Float64     │
├─────┼───────────────┼────────────┼─────────────┼─────────────┤
│ 1   │ false         │ 4.88575e-5 │ 0.98988     │ 0.986834    │

│ Row │ rank_valid_score │
│     │ Int64            │
├─────┼──────────────────┤
│ 1   │ 1                │

Best Params:
  cp => 4.8857486652379226e-5
  show_progress => false

Best Model - Fitted OptimalTreeRegressor:
  1) Split: x168 < 59.75
    2) Split: x168 < 37.17
      3) Split: x168 < 26.5
        4) Split: x168 < 19.25
          5) Split: x168 < 14.5
            6) Predict: 12.51, 13 points, error 50.36
            7) Predict: 16.98, 47 points, error 137
          8) Split: x168 < 23.5
            9) Predict: 21.74, 80 points, error 199.6
            10) Predict: 25.19, 111 points, error 223
        11) Split: x168 < 32.13
          12) Split: x168 < 29.8
            13) Predict: 28.08, 139 points, er

In [5]:
lnr = IAI.get_learner(grid)
score = IAI.score(lnr, test_X, test_y) 

# Testing data
start1 = 2019, 1, 1
ndays = 365-dayslookback-1
test_X1, test_y1 = build_multi_data(mdata, predicttype, start1, dayslookback,
 ndays; offset=offset);

start2 = 2020, 1, 1
ndays = 300 # since we don't have days until the end
test_X2, test_y2 = build_multi_data(mdata, predicttype, start2, dayslookback, 
    ndays; offset=offset)

([40.0 40.0 … 8.0 0.0; 40.0 40.0 … 8.0 1.0; … ; 53.0 53.0 … 2.0 22.0; 53.0 52.0 … 2.0 23.0], [36.6667, 36.0, 36.0, 36.0, 36.0, 35.0, 34.0, 35.0, 34.0, 36.0  …  40.0, 39.0, 38.0, 37.0, 36.0, 35.0, 35.0, 35.0, 34.0, 35.0])

In [13]:
IAI.score(lnr, train_X, train_y) 

0.9892912707266199

In [7]:
score1 = IAI.score(lnr, test_X1, test_y1)  
score2 = IAI.score(lnr, test_X2, test_y2)

0.9848122741885112

In [11]:
score, score1, score2

(0.9882827406212967, 0.9883284446212419, 0.9848122741885112)

## Analyze

In [15]:
start = 2018, 1, 2
dayslookback = 5
ndays = 365-dayslookback-1
predicttype = :temp
offsets = [1,7,13,19,25];

df = DataFrame(score_is=[], score_oos=[], score1=[], score2=[]);
for (idx, offset) in enumerate(offsets)
    X, y = build_multi_data(mdata, predicttype, start, dayslookback, ndays; offset=offset)
    (train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, X, y; seed=123)
    lnr = IAI.read_json(string("temp",offset,".json"))
    score = IAI.score(lnr, test_X, test_y) 

    # Testing data
    start1 = 2019, 1, 1
    ndays = 365-dayslookback-1
    test_X1, test_y1 = build_multi_data(mdata, predicttype, start1, dayslookback,
     ndays; offset=offset);

    start2 = 2020, 1, 1
    ndays = 300 # since we don't have days until the end
    test_X2, test_y2 = build_multi_data(mdata, predicttype, start2, dayslookback, 
        ndays; offset=offset)
    
    score_is = IAI.score(lnr, train_X, train_y)
    score_oos = IAI.score(lnr, test_X, test_y)
    score1 = IAI.score(lnr, test_X1, test_y1)  
    score2 = IAI.score(lnr, test_X2, test_y2)
    push!(df,[score_is, score_oos, score1, score2])
end

In [16]:
df

,score_is,score_oos,score1,score2
,Any,Any,Any,Any
1,0.989083,0.988628,0.988794,0.985129
2,0.898848,0.898178,0.872752,0.837184
3,0.864762,0.863635,0.834549,0.794118
4,0.853719,0.839817,0.783901,0.738273
5,0.845731,0.830434,0.761255,0.686898
